In [ ]:
import datetime as dt
from datetime import datetime

import pandas as pd
import numpy as np
import altair as alt
import holoviews as hv

from skyfield.api import load
from skyfield.api import Topos
from skyfield import almanac

from pytz import timezone

hv.extension('bokeh')

ts = load.timescale()
e = load('de421.bsp')

In [ ]:
year = 2018
place = "Paris"
point = ("48.864716 N", "2.349014 E")
up = "06:30:00"
down = "23:30:00"

In [ ]:
loc = Topos(*point)

In [ ]:
locations = [
    ("Paris", Topos(*("48.864716 N", "2.349014 E"))),
    ("Brest", Topos(*("48.4000000 N", "4.4833300 W"))),
    ("Strasbourg", Topos(*("48.5839200 N", "7.7455300 E")))
]

In [ ]:
modes = ["dst", "winter", "summer"]

In [ ]:
def compute(year, loc):
    start = ts.utc(year, 1, 1)
    end = ts.utc(year, 12, 31)
    
    t, y = almanac.find_discrete(start, end, almanac.sunrise_sunset(e, loc))
    
    sunrises = t[::2]
    sunsets = t[1::2]
    
    df = pd.DataFrame(index=range(len(sunrises)))

    def to_time(date):
        return np.datetime64(date.strftime("1970-01-01 %H:%M:%S"))

    
    df["sunrise"] = sunrises.utc_datetime()
    df["sunset"] = sunsets.utc_datetime()

    df["day"] = df["sunrise"].apply(lambda x: np.datetime64(x.strftime("%Y-%m-%d")))
    
    df["dst_sunrise"] = sunrises.astimezone(timezone("Europe/Paris"))
    df["dst_sunset"] = sunsets.astimezone(timezone("Europe/Paris"))
    
    df["dst_sunrise"] = df["dst_sunrise"].apply(lambda x: np.datetime64(x.strftime("1970-01-01 %H:%M:%S")))
    df["dst_sunset"] = df["dst_sunset"].apply(lambda x: np.datetime64(x.strftime("1970-01-01 %H:%M:%S")))
    
    df["winter_sunrise"] = df["sunrise"].apply( lambda x: to_time(x.replace(tzinfo=None)  + pd.DateOffset(hours=1)) )
    df["winter_sunset"] = df["sunset"].apply( lambda x: to_time(x.replace(tzinfo=None)  + pd.DateOffset(hours=1)) )

    df["summer_sunrise"] = df["sunrise"].apply( lambda x: to_time(x.replace(tzinfo=None) + pd.DateOffset(hours=2)) )
    df["summer_sunset"] = df["sunset"].apply( lambda x: to_time(x.replace(tzinfo=None)  + pd.DateOffset(hours=2)) )
    
    return df


df = compute(year, loc)

In [ ]:
%opts Area [bgcolor="#001f3f" xticks=3] (line_color="white" line_width=3 fill_color="#FFDC00")
%opts HLine (line_color="#7FDBFF" line_width=1)

def charts(df):

    dst, winter, summer = [ hv.Area(df, kdims=['day'], vdims=['{}_sunrise'.format(m), '{}_sunset'.format(m)], label=m) for m in modes ]

    day = (np.datetime64("1970-01-01 00:00:00"),np.datetime64("1970-01-01 23:59:59"))

    line_up = hv.HLine(np.datetime64("1970-01-01 {}".format(up)))
    line_down = hv.HLine(np.datetime64("1970-01-01 {}".format(down)))

    dst = dst.redim(dst_sunrise=hv.Dimension('dst_sunrise', range=day)).opts(width=500, height=360)
    winter = winter.redim(winter_sunrise=hv.Dimension('winter_sunrise', range=day)).opts(width=500, height=360)
    summer = summer.redim(summer_sunrise=hv.Dimension('summer_sunrise', range=day)).opts(width=500, height=360)

    return (line_up * line_down * (dst + winter + summer))

hv.output(charts(df), fig='png')

In [ ]:
def lighting(df, mode, up, down):
    e_morning = df["{}_sunrise".format(mode)] - np.datetime64("1970-01-01 {}".format(up))
    e_evening = np.datetime64("1970-01-01 {}".format(down)) - df["{}_sunset".format(mode)]
    
    e = e_morning[ e_morning > dt.timedelta(0)].sum() + e_evening[ e_evening > dt.timedelta(0)].sum()
    
    return e

def sun(df, mode, up, down):
    total = df["{}_sunset".format(mode)] - df["{}_sunrise".format(mode)]
    sleep_sun = np.datetime64("1970-01-01 {}".format(up)) - df["{}_sunrise".format(mode)]
    s = total.sum() - (sleep_sun[sleep_sun > dt.timedelta(0)].sum())
    
    return s

def summary(year, place, loc, up, down, functions):
    df = compute(year, loc)

    rows = [ (year, place, up, down, m, f[0], f[1](df, m, up, down).total_seconds(), f[1](df, m, up, down)) for m in modes for f in functions ]
    
    df_results = pd.DataFrame(rows,columns=["year", "place", "up", "down", "mode", "energy", "seconds", "timedelta"])

    return df_results

comp = pd.concat([ summary(year, place, loc, up, down, [("sun", sun), ("lighting", lighting)]) for place, loc in locations ])

comp

In [ ]:
def diff(df, ref, var):
    gr = df[ df["mode"].isin(ref) ].groupby(["year", "place", "up", "down", "energy"])
    gv = df[ df["mode"].isin(var) ].groupby(["year", "place", "up", "down", "energy"])
 
    def dst_delta(n,g):
        dst = gr.get_group(n)["timedelta"]
        delta = g.apply(lambda x: x["timedelta"] - dst, axis=1)
    
        return pd.concat([g, pd.Series(delta[list(delta)[0]],name="dst_delta")], axis=1)
    
    
    return [ dst_delta(n,g) for n,g in gv ]

    
delta = pd.concat(diff(comp, ["dst"], ["dst","winter", "summer"]))

delta["dst_delta_seconds"] = delta.apply(lambda x: x['dst_delta'].total_seconds(), axis=1)

delta

In [ ]:
delta.pivot_table(index="mode",columns=["place", "energy"], values=["dst_delta_seconds"])

In [ ]:
hv.Layout([ hv.Bars(delta[delta['place'] == place ], ['mode', 'energy'], 'dst_delta_seconds', label=place).opts(padding=0.2) for place, loc in locations ])

In [ ]:
import panel as pn
import param
from holoviews.streams import Params

In [ ]:
class DSTCalc(param.Parameterized):    
    place = param.ObjectSelector(default="Paris", objects=[ p for p,l in locations ])
    
    up = param.Date()
    down = param.Date()

    @param.depends('place')
    def update(self):        
        df = compute(2018, [ l for p,l in locations if p == self.place][0])

        return hv.Layout(charts(df))
    
    def panel(self):
        return pn.Column(self.param, self.update)

calc = DSTCalc()
calc.panel()